In [ ]:
# Exploring Auction Data Chile
# Sebastian Arechaga

In [205]:
#Libraries
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [225]:
# Import the data
df_lic = pd.read_table('lic_2017-1.txt' , sep='\t', encoding="latin-1")

C:\Users\sarec\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (26,51,62,63,86,97,101,102) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [224]:
# Keepinf only aucions between 100 and 1000 UTM, which have a final result (adjudicadas) and are not "Obras"
df_lic = df_lic[(df_lic.Estado == "Adjudicada") & (df_lic.Tipo == "LE") & (df_lic.Obras == 0)]

In [208]:
# Number of distinct providers (rut)
print("The number of distinct providers is" , len(set(df_lic.RutProveedor)) , ", and they are spread in a total of" , len(set(df_lic.index)) , "observations")

The number of distinct providers is 5678 , and they are spread in a total of 102994 observations


In [231]:
# Create a dataframe with 12 months of auctions with the previous filters

df_lic = df_lic[(df_lic.Estado == "Adjudicada") & (df_lic.Tipo == "LE") & (df_lic.Obras == 0)]

df_lic['Month'] = 1


for i in range(2,13):
    dfaux = pd.read_table('lic_2017-{x}.txt'.format(x=i) , sep='\t', encoding="latin-1")
    dfaux = dfaux[(dfaux.Estado == "Adjudicada") & (dfaux.Tipo == "LE") & (dfaux.Obras == 0)]
    dfaux['Month'] = i
    df_lic=df_lic.append(dfaux)
    

C:\Users\sarec\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (26,51,62,63,64,72,86,94,97,101) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\sarec\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (26,51,62,63,72,86,94,97,100,101,102) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\sarec\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (26,51,62,63,86,94,97,101) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\sarec\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (26,51,62,63,86,97,101,102) have mixed types.Specify dtype option on 

In [232]:
df_lic

,Codigo,Link,CodigoExterno,Nombre,Descripcion,Tipo de Adquisición,CodigoEstado,Estado,CodigoOrganismo,NombreOrganismo,...,Estado Oferta,Cantidad Ofertada,Moneda de la Oferta,MontoUnitarioOferta,Valor Total Ofertado,CantidadAdjudicada,MontoLineaAdjudica,FechaEnvioOferta,Oferta seleccionada,Month
108,7958276,http://www.mercadopublico.cl/fichaLicitacion.h...,2105-179-LE16,CONCESION DE CASINO DEL HOSPITAL DE LA SERENA,Contratar concesionario para el arriendo y adm...,Licitación Pública Entre 100 y 1000 UTM (LE),8,Adjudicada,7312,Hospital La Serena,...,Aceptada,1,Peso Chileno,2499,2499,1,2499,03-02-2017,Seleccionada,1
109,7958276,http://www.mercadopublico.cl/fichaLicitacion.h...,2105-179-LE16,CONCESION DE CASINO DEL HOSPITAL DE LA SERENA,Contratar concesionario para el arriendo y adm...,Licitación Pública Entre 100 y 1000 UTM (LE),8,Adjudicada,7312,Hospital La Serena,...,Aceptada,1,Peso Chileno,4500,4500,1,4500,03-02-2017,Seleccionada,1
110,7958276,http://www.mercadopublico.cl/fichaLicitacion.h...,2105-179-LE16,CONCESION DE CASINO DEL HOSPITAL DE LA SERENA,Contratar concesionario para el arriendo y adm...,Licitación Pública Entre 100 y 1000 UTM (LE),8,Adjudicada,7312,Hospital La Serena,...,Aceptada,1,Peso Chileno,700,700,1,700,03-02-2017,Seleccionada,1
111,7958276,http://www.mercadopublico.cl/fichaLicitacion.h...,2105-179-LE16,CONCESION DE CASINO DEL HOSPITAL DE LA SERENA,Contratar concesionario para el arriendo y adm...,Licitación Pública Entre 100 y 1000 UTM (LE),8,Adjudicada,7312,Hospital La Serena,...,Aceptada,1,Peso Chileno,4.00E+05,4.00E+05,1,4.00E+05,03-02-2017,Seleccionada,1
254,7984371,http://www.mercadopublico.cl/fichaLicitacion.h...,2354-17-LE16,SUMINISTRO IMPLEMENTOS DEPORTIVOS,SUMINISTRAR DE IMPLEMENTACIÓN DEPORTIVA,Licitación Pública Entre 100 y 1000 UTM (LE),8,Adjudicada,86580,I MUNICIPALIDAD DE PUNTA ARENAS,...,Aceptada,1,Peso Chileno,"24789,91","24789,91",1,"24789,91",10-01-2017,Seleccionada,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186810,8405631,http://www.mercadopublico.cl/fichaLicitacion.h...,2446-544-R118,Licitación Pública Nº 189/2017 ADQUISICIÓN E ...,Licitación Pública que comprende la adquisició...,Licitación Pública Entre 100 y 1000 UTM (LE),8,Adjudicada,87901,I. MUNICIPALIDAD DE COQUIMBO,...,Aceptada,1,Peso Chileno,3400000,3400000,0,0,11-12-2017,No Seleccionada,12
186811,8405631,http://www.mercadopublico.cl/fichaLicitacion.h...,2446-544-R118,Licitación Pública Nº 189/2017 ADQUISICIÓN E ...,Licitación Pública que comprende la adquisició...,Licitación Pública Entre 100 y 1000 UTM (LE),8,Adjudicada,87901,I. MUNICIPALIDAD DE COQUIMBO,...,Aceptada,1,Peso Chileno,4287572,4287572,0,0,13-12-2017,No Seleccionada,12
186812,8405631,http://www.mercadopublico.cl/fichaLicitacion.h...,2446-544-R118,Licitación Pública Nº 189/2017 ADQUISICIÓN E ...,Licitación Pública que comprende la adquisició...,Licitación Pública Entre 100 y 1000 UTM (LE),8,Adjudicada,87901,I. MUNICIPALIDAD DE COQUIMBO,...,Aceptada,1,Peso Chileno,2268068,2268068,0,0,13-12-2017,No Seleccionada,12
186813,8405631,http://www.mercadopublico.cl/fichaLicitacion.h...,2446-544-R118,Licitación Pública Nº 189/2017 ADQUISICIÓN E ...,Licitación Pública que comprende la adquisició...,Licitación Pública Entre 100 y 1000 UTM (LE),8,Adjudicada,87901,I. MUNICIPALIDAD DE COQUIMBO,...,Aceptada,1,Peso Chileno,3890000,3890000,0,0,14-12-2017,No Seleccionada,12
